# Data Cleaning for NBA Fantasy Points

In [9]:
import pandas as pd
import os
from glob import glob

In [10]:
%run cleaning_functions.py

In [11]:
# Path to data
GAME_PATH = '.\\..\\..\\web-scraping\\game-data'
FANTASY_PATH = '.\\..\\..\\web-scraping\\fantasy-data'
OUTPUT = '.\\clean-data'

### Saving All .csv File Paths

In [12]:
game_files = []
for path, subdir, files in os.walk(GAME_PATH):
    for file in glob(os.path.join(path, '*.csv')):
        game_files.append(file)

In [13]:
fantasy_files = []
for path, subdir, files in os.walk(FANTASY_PATH):
    for file in glob(os.path.join(path, '*.csv')):
        fantasy_files.append(file)

### Cleaning Data Frame

In [14]:
# drop useless columns
raw_df_fantasy = pd.concat(
    map(pd.read_csv, fantasy_files), ignore_index=True)
drop = ['playerId', 'rosterPosition', 'currentTeamId', 'eventId', 'eventTeamId', 'homeVisitor','favDog','statDetails', 'madeCut']
df_fantasy = raw_df_fantasy.drop(columns = drop)

In [15]:
raw_df_game = pd.concat(
    map(pd.read_csv, game_files), ignore_index=True)
drop = []
df_game = raw_df_game.drop(columns=drop)

In [16]:
# change DNP data to 0
attempt_columns = ['FG','3PT', 'FT']
point_columns = ['MIN','OREB','DREB','REB','AST','STL','BLK','TO','PF','+/-','PTS']
df_game[attempt_columns] = df_game[attempt_columns].replace(to_replace=r'DNP.*', value='0-0', regex=True)
df_game[point_columns] = df_game[point_columns].replace(to_replace=r'DNP.*', value=0, regex=True)
# split columns with made and attempted shots
for column in attempt_columns:
    df_game[[column + 'M', column + 'A']] = df_game[column].str.split('-', 1, expand=True)
df_game = df_game.drop(columns = attempt_columns)
df_game = df_game.replace('O.G. Anunoby', 'O. Anunoby')
df_game

,Date,Player,Team,MIN,OREB,DREB,REB,AST,STL,BLK,TO,PF,+/-,PTS,FGM,FGA,3PTM,3PTA,FTM,FTA
0,2022-11-01,P. Williams,CHI,30,3,4,7,1,1,2,3,2,-8,12,5,10,0,3,2,2
1,2022-11-01,D. DeRozan,CHI,33,0,4,4,1,3,0,1,2,-1,20,8,21,1,2,3,3
2,2022-11-01,N. Vucevic,CHI,32,3,12,15,2,1,0,1,2,+5,7,3,8,0,1,1,2
3,2022-11-01,Z. LaVine,CHI,37,1,2,3,5,0,0,1,2,+7,29,10,21,5,11,4,4
4,2022-11-01,A. Dosunmu,CHI,33,0,3,3,4,3,0,2,2,+12,17,7,11,1,4,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5193,2022-11-28,A. Len,SAC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5194,2022-11-28,R. Holmes,SAC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5195,2022-11-28,N. Queta,SAC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5196,2022-11-28,M. Dellavedova,SAC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
# Create new column to make joining easier
df_fantasy['initialName'] = initialize_name(df_fantasy['firstName'], df_fantasy['lastName'])
df_fantasy

,firstName,lastName,fullName,salary,position,currentTeam,projPoints,ownership,actualPoints,Date,initialName
0,Chris,Paul,Chris Paul,7700,PG,PHX,38.61,10.58,50.50,2022-11-01,C. Paul
1,D'Angelo,Russell,D'Angelo Russell,7000,PG,MIN,30.56,8.16,14.50,2022-11-01,D. Russell
2,Karl-Anthony,Towns,Karl-Anthony Towns,8800,C,MIN,42.59,9.39,50.00,2022-11-01,K. Towns
3,Devin,Booker,Devin Booker,9000,SG,PHX,44.76,18.76,36.25,2022-11-01,D. Booker
4,Cameron,Payne,Cameron Payne,4600,PG,PHX,18.45,4.29,15.50,2022-11-01,C. Payne
...,...,...,...,...,...,...,...,...,...,...,...
6364,Johnny,Davis,Johnny Davis,3000,SG,WAS,0.00,0.00,3.00,2022-11-28,J. Davis
6365,Trevor,Hudgins,Trevor Hudgins,3000,PG,HOU,0.00,0.01,0.00,2022-11-28,T. Hudgins
6366,Dyson,Daniels,Dyson Daniels,3300,PG,NOP,18.33,8.00,21.75,2022-11-28,D. Daniels
6367,Ousmane,Dieng,Ousmane Dieng,3000,SF,OKC,7.00,0.00,16.25,2022-11-28,O. Dieng


### Joining Tables

In [18]:
df = df_fantasy.merge(df_game, how='inner',left_on=['initialName','currentTeam','Date'], right_on=['Player','Team','Date'])
df = df.drop(columns=['initialName','currentTeam','Player','firstName', 'lastName'])
df

,fullName,salary,position,projPoints,ownership,actualPoints,Date,Team,MIN,OREB,...,TO,PF,+/-,PTS,FGM,FGA,3PTM,3PTA,FTM,FTA
0,Chris Paul,7700,PG,38.61,10.58,50.50,2022-11-01,PHX,34,1,...,1,2,+18,15,5,12,1,5,4,6
1,D'Angelo Russell,7000,PG,30.56,8.16,14.50,2022-11-01,MIN,23,0,...,4,3,-20,5,2,8,1,3,0,0
2,Karl-Anthony Towns,8800,C,42.59,9.39,50.00,2022-11-01,MIN,37,5,...,4,4,-12,24,9,18,3,7,3,3
3,Devin Booker,9000,SG,44.76,18.76,36.25,2022-11-01,PHX,30,1,...,2,5,+11,18,6,18,2,7,4,5
4,Cameron Payne,4600,PG,18.45,4.29,15.50,2022-11-01,PHX,14,0,...,2,1,-9,8,3,8,2,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680,Jabari Smith Jr.,6000,PF,26.83,3.38,15.25,2022-11-28,HOU,17,1,...,1,3,-2,9,3,4,3,4,0,0
4681,Johnny Davis,3000,SG,0.00,0.00,3.00,2022-11-28,WAS,4,0,...,0,3,-11,3,1,3,0,1,1,1
4682,Dyson Daniels,3300,PG,18.33,8.00,21.75,2022-11-28,NOP,28,0,...,1,3,-4,8,3,5,2,3,0,0
4683,Ousmane Dieng,3000,SF,7.00,0.00,16.25,2022-11-28,OKC,11,0,...,1,2,-1,6,2,4,2,3,0,0


### Converting Column Data Types

In [19]:
to_int = ['salary', 'MIN','OREB','DREB','REB','AST','STL','BLK','TO','PF','+/-','PTS', 'FGA', 'FGM', '3PTA', '3PTM', 'FTA', 'FTM']
to_float = ['projPoints', 'ownership', 'actualPoints']
df[to_int] = df[to_int].apply(pd.to_numeric)
df[to_float] = df[to_float].astype('float64')
df['Date'] = df['Date'].astype('datetime64[ns]')
df

,fullName,salary,position,projPoints,ownership,actualPoints,Date,Team,MIN,OREB,...,TO,PF,+/-,PTS,FGM,FGA,3PTM,3PTA,FTM,FTA
0,Chris Paul,7700,PG,38.61,10.58,50.50,2022-11-01,PHX,34,1,...,1,2,18,15,5,12,1,5,4,6
1,D'Angelo Russell,7000,PG,30.56,8.16,14.50,2022-11-01,MIN,23,0,...,4,3,-20,5,2,8,1,3,0,0
2,Karl-Anthony Towns,8800,C,42.59,9.39,50.00,2022-11-01,MIN,37,5,...,4,4,-12,24,9,18,3,7,3,3
3,Devin Booker,9000,SG,44.76,18.76,36.25,2022-11-01,PHX,30,1,...,2,5,11,18,6,18,2,7,4,5
4,Cameron Payne,4600,PG,18.45,4.29,15.50,2022-11-01,PHX,14,0,...,2,1,-9,8,3,8,2,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680,Jabari Smith Jr.,6000,PF,26.83,3.38,15.25,2022-11-28,HOU,17,1,...,1,3,-2,9,3,4,3,4,0,0
4681,Johnny Davis,3000,SG,0.00,0.00,3.00,2022-11-28,WAS,4,0,...,0,3,-11,3,1,3,0,1,1,1
4682,Dyson Daniels,3300,PG,18.33,8.00,21.75,2022-11-28,NOP,28,0,...,1,3,-4,8,3,5,2,3,0,0
4683,Ousmane Dieng,3000,SF,7.00,0.00,16.25,2022-11-28,OKC,11,0,...,1,2,-1,6,2,4,2,3,0,0


### Ouputting Data

In [20]:
# rename headers
df.rename(columns={'fullName': 'Player', 'salary': 'Salary', 'position': 'Position', 'projPoints': 'Projected Points', 'ownership': 'Ownership', 'actualPoints': 'Actual Points'}, inplace=True)

df_game.to_csv(OUTPUT + '\\game-table.csv',',', index=False)
df_fantasy.to_csv(OUTPUT + '\\fantasy-table.csv', ',', index=False)
df.to_csv(OUTPUT + '\\clean-data.csv',',', index=False)